In [2]:

#!/usr/bin/env python
# coding: utf-8

import PySimpleGUI as sg
import pandas as pd
import datetime
import os
import re

import pyodbc
import pandas.io.sql as psql
import win32com.client



def create_report():

    # from datetime import datetime
    print('старт')
    text_elem = window['-text-']
    text_elem.update("Отчет формируется, не нажимать")
    
    sql = '''
    SET NOCOUNT ON;
    
    --1
    SELECT * INTO #ak_work_table
    FROM
    (
    select  --COUNT(*)
            RQ.id				AS RQ_ID
            ,rg.NKOD			AS reglament
            ,s.nkod				AS status_
            ,rq.Date_Request    AS dt
            ,rq.Date_Export     AS Date_Export

    from SYST_REQUEST rq (nolock)
    join f6_SYST_request f6s  (nolock)on f6s.SYST_request_id=rq.id
    join f6  (nolock)on f6.id=f6s.f6_id and DATRZ>='01.04.2021' and (KOD=1022700000000 )
    join SYST_REGLAM rg  (nolock)on rq.SYST_REGLAM_ID=rg.ID 
    left join SYST_QUERIES q  (nolock)on q.SYST_REQUEST_ID=rq.id
    left join SYST_STATUS s  (nolock) on s.id=q.state

    UNION
    --2
    SELECT   
            --COUNT(*)
            SRQ.ID
            , SR.NKOD
            , s.nkod
            , SRQ.Date_Request
            ,SRQ.Date_Export

    FROM SYST_REQUEST (nolock) SRQ
    join SYST_REGLAM (nolock) SR 
    on SRQ.SYST_REGLAM_ID = sr.id 
        --AND NKOD  LIKE  'ЗАГС%'
        --and sr.ID In (1431,1432,1433,1434,1390) 
        and User_Request in (22,327)
    left join SYST_QUERIES (nolock) q on q.SYST_REQUEST_ID=SRQ.id
    left join SYST_STATUS (nolock) s  on s.id=q.state
    WHERE SRQ.Date_Request>'01.04.2021'

    UNION
    --3
    select  
            RQ.id				AS RQ_ID
            ,rg.NKOD			AS reglament
            ,s.nkod				AS status_
            ,rq.Date_Request    AS dt
            ,rq.Date_Export    

    from SYST_REQUEST rq (nolock)
    join SYST_REGLAM rg  (nolock) on rq.SYST_REGLAM_ID=rg.ID --and NKOD  LIKE  '3834R%'
    left join SYST_QUERIES q  (nolock) on q.SYST_REQUEST_ID=rq.id
    left join SYST_STATUS s  (nolock) on s.id=q.state

    where Date_Request >'01.04.2021' and f2_id in 
    (select p.f2_id from f_pensfam p
    join f6  (nolock)on f6.id=p.decl_id and f6.DATRZ>='01.04.2021' and f6.KOD=1022700000000)
    ) T1;


    --Уникальные значения. Скачить их в CSV формате. 
    --скрипт учитывает что скачается без заголовка
    SELECT DISTINCT T1.RQ_ID, 
                    T1.reglament, 
                    T1.status_, 
                    dt, 
                    iif (RQ_ID in (select SYST_REQUEST_ID from SYST_QUERIES), 1, 0)      AS [answ], 
                    iif (RQ_ID in (select SYST_request_id from [SYST_wait_param]), 1, 0) AS SYST_wait, 
                    iif (T1.Date_Export is not null, 1, 0)                               AS Date_Export
    from #ak_work_table T1
    WHERE CONCAT(T1.RQ_ID, dt) in 
                                (SELECT CONCAT(#ak_work_table.RQ_ID, max(dt))
                                 FROM #ak_work_table
                                 group by RQ_ID);


    '''

    import pyodbc
    
    cnxn = pyodbc.connect('DRIVER={SQL Server}; SERVER=oirc-srv-sql15; DATABASE=SVO_ETL; Trusted_Connection=True')
#     cnxn = pyodbc.connect('DRIVER={SQL Server}; SERVER=oirc-srv-test03; DATABASE=EVD_SVO_ETL; Trusted_Connection=True')
    cursor = cnxn.cursor()

#     %%time

    df_0 = pd.read_sql(sql, cnxn)
    cnxn.close()

    df_0.columns = ['RQ_ID', 'reglament', 'status_', 'dt', 'answ', 'SYST_wait', 'Date_Export']
    
    print(df_0)
    # Ищим опоздавшие ____ запросы

    # добавляем столбец для обозначающий время за которое должен быть быть обработан запрос
    df_0['dt'] = pd.to_datetime(df_0['dt'])
    df_0['dt_end'] = df_0.dt + pd.Timedelta(days=7)

    # берём дату сегодня
    to_day = datetime.datetime.today()

    df_0['chack_late'] = (df_0['dt_end'] < to_day)
    
    df_0['chack_late_1_0']  = df_0['chack_late'].astype(int) & (df_0['status_'].apply(lambda x: 'Ожидание запроса службы' in str(x))).astype(int)

    # переназначаю поле 
    df_0['Date_Export'] = ((df_0['SYST_wait'] == 0) & (df_0['Date_Export'] == 0))

    # переназначаем поле 
    df_0['answ']        = ((df_0['SYST_wait'] == 0) & (df_0['answ'] == 0))


    # Убираю этот регламент записи 
    # обновление 30.06.2021
    df_0 = df_0.loc[df_0['reglament'] != 'Занятость_39....', :]
    

    m_list = []
    def make_cut_reglam(name):
        global mvd_df
        name = str(name)
        
        #регламент нужно сортировать по разделам. Каждый относится с каждому. 
        #для группировки беру срез в ситуации если первые 3 букрвы названия не ранвы ___
        #Если равны нужно брать больше
        if  name[:3] != "___": 
            name = name[:4]
        else:
            m_list.append(name)

            name = name[:25]
        return  name

    df_0['название'] = df_0.reglament.apply(make_cut_reglam)

    # Добавляю условие Ф__ считаю с 01.06.2021

    df_0 = df_0.loc[~((df_0['название'] == "Ф__")&(df_0['dt_end'] < datetime.datetime(2021, 6, 1)))]

    # Пивоты

    df_piv = pd.pivot_table(data=df_0, index='название', columns='status_', values='RQ_ID', aggfunc='count').reset_index()

    #Делаю второй пивот

    df_piv_answ_2 = pd.pivot_table(data=df_0, index='название', columns='answ', values='RQ_ID', aggfunc='count').reset_index()                      .rename({True:'Не_ответили', False:'Ответили'}, axis=1)

    #Делаю третий пивот статистику по опозданиям 

    df_piv_late_3 = pd.pivot_table(data=df_0, index='название', columns='chack_late_1_0', values='RQ_ID', aggfunc='count').reset_index()                    .rename({0:'Не_опоздал', 1:'Опоздал'}, axis=1)

    #Делаю четвёртый пивот

    df_piv_late_4 = pd.pivot_table(data=df_0, index='название', 
                                   columns='SYST_wait', values='RQ_ID', 
                                   aggfunc='count')\
                        .reset_index()\
                        .rename({1:'smew_wait', 0:'not_in_smew_wait_param'}, axis=1)

    #Делаю пятый пивот 

    df_piv_late_5 = pd.pivot_table(data=df_0, index='название', 
                                   columns='Date_Export', values='RQ_ID', 
                                   aggfunc='count')\
                        .reset_index()\
                        .rename({0:'has_date_export', 1:'waiting_export_of_ASP'}, axis=1)
    # waiting_export_of_ASP = не имеющие дату выгрузки

    df_0['Date_Export'].value_counts()

    # 1 пивот
    df = df_piv.loc[:]
    df['название_0'] = df['название']
    df = df_piv.groupby('название', as_index=False).sum()

    # 2 пивот
    df_2_2 = df_piv_answ_2.loc[:]
    df_2_2 = df_piv_answ_2.groupby('название', as_index=False).sum()

    # 3 пивот
    df_2_3 = df_piv_late_3.loc[:]
    df_2_3 = df_piv_late_3.groupby('название', as_index=False).sum()

    # 4 пивот 
    df_2_4 = df_piv_late_4.loc[:]
    df_2_4 = df_piv_late_4.groupby('название', as_index=False).sum()

    # 5 пивот
    df_2_5 = df_piv_late_5.loc[:]
    df_2_5 = df_piv_late_5.groupby('название', as_index=False).sum()

    df = df.merge(df_2_2 , on='название', how='left')
    df = df.merge(df_2_3, on='название', how='left')
    df = df.merge(df_2_4, on='название', how='left')
    df = df.merge(df_2_5, on='название', how='left')

    df = df.loc[df['название'] != ' ___мото транспорте', :]
    # обновление 17.05.2021
    #': ___мото транспорте' 
    # строка удалена. Сказано не учитывать её 

    

    

# переименования
    df.replace({'название':{"страрое":'новое'}}
                           , inplace=True)

    #удалаяем лишние символы из готовых названий регламентов (возможно уже все названия регламентов переделаны с от начала до конца, тогда это действие не нужно) 

    def drop_(name):
        name =str(name)
        name = name.replace('_', '')
        name = name.replace('-', '')
        return name

    df['название'] = df['название'].apply(drop_)

    df_group = df.groupby('название', as_index=False).sum()

    df_itog = pd.DataFrame()

    
#     урезання нформация о названиях колонок
    df_itog['названия рег ____'] = df_group.loc[:, 'название']

    df_itog['Количество ____ запросов всего'] = df_group['Не_ответили'] + df_group['Ответили']

    df_itog['Количество полученных ответов'] = df_group['Ответили'] - df_group['Ожидание запроса службы ____ №2']

    df_itog['Данные помещены в базу данных.'] =  df_group['Завершение опроса служб ____']

    df_itog['Положительные.  ____ вернула результат: данные не найдены'] = df_group['Получен ответ от службы ____ №1 (данные не найдены)'] +                                                                                  df_group['Получен ответ от службы ____ №2 (данные не найдены)']   


    df_itog['Получен ответ с сообщением об ошибке  ____'] =  df_group['Отменен запрос к службе ____ №1'] +                 df_group['Получен ответ с сообщением об ошибке от службы ____ №1'] +                 df_group['Получен ответ с сообщением об ошибке от службы ____ №2']

    df_itog['Находятся на рассмотрении, всего']  = df_group['Ожидание запроса службы ____ №2']
    
    df_itog['Ожидающи'] = df_group['waiting_export_of_ASP']
    
    df_itog['нарушение срока'] = df_group['Опоздал']
    
    df_itog['ответы не загружены '] = df_group['Не_ответили'] - df_group['waiting_export_of_ASP']
    
    df_itog['название 1'] = df_group['smew_wait'] 

    df_itog_2 = df_itog.iloc[:, :]

    to_day = to_day.strftime("%d-%m-%Y")

#     file_path = f'F:/ak.ardyshev/ЗАДАЧИ/3 Отчеты по ____ запросам/Отчет/Информация по ____ запросам c 01-04-2021 по {to_day}.xlsx'
    file_name = f'Информация по ____ запросам c 01-04-2021 по {to_day}.xlsx'
    
    file_path = f'F:/ak.ardyshev/ЗАДАЧИ/3 Отчеты по ____ запросам/Отчет/Информация по ____ запросам c 01-04-2021 по {to_day}.xlsx'
    
    file_location = 'F:/ak.ardyshev/ЗАДАЧИ/3 Отчеты по ____ запросам/Отчет/'
    
    if os.path.isdir(file_location) is True:
        file_path = file_location + file_name
    else:
        file_path = path + file_name
    
    df_itog_2.to_excel(file_path, header=False, startrow=2, index=False)

    # Стандарт 16 (на дату 26.05.2021)

    # Для красоты


    writer = pd.ExcelWriter(file_path,
                            engine='xlsxwriter')

    df_itog_2.to_excel(writer, sheet_name='Sheet1', startrow=1, index=False)
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    # Вписываю заголовки
    worksheet.write_row('A2', list(df_itog_2))



    # устанавливаю ширину и высоту
    worksheet.set_row(1, 90)
    worksheet.set_column(0, 0, 50)
    for i in range(1, len(list(df_itog_2))):
        worksheet.set_column(i, i, 18)


    merge_format = workbook.add_format({
        'bold':     True,
        'border':   6,
        'align':    'center',
        'valign':   'vcenter',
        'fg_color': '#DCDCDC',
    })
    worksheet.merge_range('A1:K1', F'Информация по запросам ____  за период с 01-04-2021 по {to_day}', merge_format)


    writer.save()

    from openpyxl.utils.cell import get_column_letter
    from openpyxl import load_workbook
    from openpyxl.styles import Alignment
    from openpyxl.styles import Border
    from openpyxl.styles import Side

    wb = load_workbook(file_path)
    ws = wb.worksheets[0]


    thin = Side(border_style='thin', color='000000')

    num_row = 0
    for row in ws.iter_rows(min_row=2, min_col=1, max_col=11):
        num_row += 1
        for cell in row:
            cell.alignment = Alignment(horizontal='center', vertical='center', wrapText=True)
            cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

    max_row =    ws.max_row
    max_column = ws.max_column

    for col_num in range(2, max_column+1):
        name_col = get_column_letter(col_num)
        ws[f"{name_col}{max_row+1}"] = f'=SUM({name_col}{3}:{name_col}{max_row})'
        # Расчитываю сумму. Пример: ws["B19"] = "=SUM(B3:B18)"

    for row in ws.iter_rows(min_row=3, min_col=2, max_col=11):
        for cell in row:
             cell.number_format = '#,##0'

    wb.save(file_path)
    
    done = True
    text_elem.update(f"Отчет сформирован можете выходить  done={done}")

    if not values['-email-']:
        return 1
#     Отправка писем
    email_addresses = ['kirill.arda@yandex.ru']

    outlook = win32com.client.Dispatch("Outlook.Application")

    attachment_path = file_path

    for email in email_addresses:
        mail = outlook.CreateItem(0)
        mail.To = email
        mail.Subject = f'____ {to_day}'           #Тема
        mail.Body = """Отчет по ____ запросам """ #Текст
        mail.Attachments.Add(Source=attachment_path)
        mail.Send()
        
        
        
        
layout = [[sg.Input(), sg.FolderBrowse('FolderBrowse')],
          [sg.Button('Generate',enable_events=True, key='-FUNCTION-', font='Helvetica 16'),
          sg.CB('send email', key='-email-', change_submits=True, default=True)],
          [sg.Text('Нажмите для формирования. В диспечере будет отображаться "Файл не отвечает", выполнятся может до 25 мин.', size=(60, 3), key='-text-', font='Helvetica 16')]]
layout2 = [
    [sg.Text('Выберети папку!', size=(35, 1))]
]

# Create the window
window = sg.Window('create report ____', layout)
window1 = sg.Window('Ошибка', layout2)
# Event loop
flag_num = True
while True:
    event, values = window.read()
    if event in (sg.WIN_CLOSED, 'Exit'):
        break
        
    path = values['FolderBrowse']
    path_true = re.search(r'\S', path)
    
    file_location = 'F:/ak.ardyshev/ЗАДАЧИ/3 Отчеты по ____ запросам/Отчет/'
    if os.path.isdir(file_location) is True:
        path_true = True
    
    if event == '-FUNCTION-' and path_true is not None and flag_num == True:
        flag_num = False
        create_report()
    elif event == '-FUNCTION-':
        event2 = window1.read()
        if event2 in (sg.WIN_CLOSED, 'Exit'):
            break

# Close the window i.e. release resource
window.close()

Пример в котором я скачиваю данны их SQL

(Пока не сделан с помощью SQL pivot т.к. добавляются новые регламенты, в SQL pivot я не знаю как это учесть.)

install_data.jpg

<img src= install_data.jpg\>

In [ ]:
Пример формы в которой я получаю отчет 

push_data

<img src= push_data.jpg\>